In [35]:
%pip -q install PyMySQL cryptography gensim


After uploading the .pem file, we connect to that database.

In [36]:
import pymysql, os

DB_CONFIG = {
    "host": os.environ["DB_HOST"],
    "port": int(os.environ.get("DB_PORT", "3306")),
    "user": os.environ["DB_USER"],
    "password": os.environ["DB_PASSWORD"],
    "db": os.environ["DB_NAME"],
    "charset": "utf8mb4",
    "cursorclass": pymysql.cursors.DictCursor,
    "connect_timeout": 10,
}

#connection test
conn = pymysql.connect(**DB_CONFIG)
with conn.cursor() as cur:
    cur.execute("SELECT VERSION() AS version, NOW() AS server_time;")
    row = cur.fetchone()
    print("Connected to MySQL", row["version"], "| Server time:", row["server_time"])
conn.close()

Connected to MySQL 8.0.35 | Server time: 2026-01-19 22:39:54


In [37]:
conn = pymysql.connect(**DB_CONFIG)
cur = conn.cursor()
cur.execute("DROP TABLE IF EXISTS movies;")
conn.commit()
conn.close()
print("Dropped old movies table.")

Dropped old movies table.


In [38]:
conn = pymysql.connect(**DB_CONFIG)
with conn.cursor() as cur:
    #create movies table if missing
    cur.execute("""
        CREATE TABLE IF NOT EXISTS movies (
            id INT AUTO_INCREMENT PRIMARY KEY,
            title VARCHAR(255) NOT NULL,
            plot  TEXT,
            UNIQUE KEY uq_movies_title (title)
        ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
    """)

    #add vector column if it doesn’t exist
    cur.execute("""
        SELECT COUNT(*) AS n
        FROM INFORMATION_SCHEMA.COLUMNS
        WHERE TABLE_SCHEMA = DATABASE()
          AND TABLE_NAME = 'movies'
          AND COLUMN_NAME = 'vec_json';
    """)
    has_vec = cur.fetchone()["n"] > 0
    if not has_vec:
        cur.execute("ALTER TABLE movies ADD COLUMN vec_json LONGTEXT;")
        print("added movies.vec_json column.")
    else:
        print("movies.vec_json already present.")
conn.commit()
conn.close()

added movies.vec_json column.


In [39]:
movies_to_add = [
    #Sci fi and action
    ("Inception", "A thief enters dreams to steal secrets from the subconscious."),
    ("Interstellar", "Astronauts travel through a wormhole to save humanity."),
    ("The Matrix", "A hacker discovers reality is a simulation controlled by machines."),
    ("Star Wars: A New Hope", "Rebels battle the Galactic Empire in a distant galaxy."),
    ("The Terminator", "A cyborg assassin travels through time to kill a woman whose son will save humanity."),
    ("Blade Runner", "A detective hunts synthetic humans in a futuristic city."),
    ("Avatar", "Humans explore and exploit a lush alien world called Pandora."),
    ("Dune", "A noble family becomes embroiled in a war over a desert planet."),
    ("The Martian", "An astronaut stranded on Mars must survive alone."),
    ("Guardians of the Galaxy", "A group of misfits band together to save the universe."),

    # romance and drame
    ("Titanic", "A love story unfolds aboard the doomed ocean liner."),
    ("The Notebook", "Two lovers are separated by class and time but never forget each other."),
    ("La La Land", "A musician and an actress fall in love while chasing their dreams."),
    ("Pride and Prejudice", "A young woman challenges social class to find true love."),
    ("The Fault in Our Stars", "Two teens with cancer fall in love while facing mortality."),
    ("Notting Hill", "A famous actress falls for an ordinary bookshop owner."),
    ("The Holiday", "Two women swap homes and find unexpected love."),
    ("A Star Is Born", "A musician helps a young singer find fame as he declines."),
    ("Crazy Rich Asians", "A woman discovers her boyfriend is from a wealthy and powerful family."),
    ("Me Before You", "A young woman cares for a paralyzed man and they fall in love."),

    # crime and thriller
    ("The Godfather", "A crime boss passes control of his empire to his reluctant son."),
    ("Goodfellas", "A mob associate rises and falls in the mafia world."),
    ("Pulp Fiction", "Criminals' lives intertwine in violent and darkly comic ways."),
    ("The Dark Knight", "Batman faces chaos as the Joker terrorizes Gotham City."),
    ("Heat", "A detective and a master thief face off in Los Angeles."),
    ("Se7en", "Detectives hunt a serial killer who uses the seven deadly sins."),
    ("The Departed", "An undercover cop and a mole try to expose each other in the police force."),
    ("Scarface", "A Cuban immigrant rises to power in Miami’s brutal drug trade."),
    ("Casino", "A casino manager faces mob pressure and personal betrayal."),
    ("John Wick", "A retired hitman seeks revenge for the death of his beloved dog."),
]

conn = pymysql.connect(**DB_CONFIG)
with conn.cursor() as cur:
    cur.executemany("""
        INSERT INTO movies (title, plot)
        VALUES (%s, %s)
        ON DUPLICATE KEY UPDATE plot = VALUES(plot);
    """, movies_to_add)
conn.commit()
conn.close()
print(f"updated {len(movies_to_add)} movies without duplicates.")

updated 30 movies without duplicates.


In [40]:
import re, json, numpy as np
import gensim.downloader as api
from numpy.linalg import norm

#load pretrained 100D embeddings (cached after first run)
kv = api.load("glove-wiki-gigaword-100")
EMBED_DIM = kv.vector_size

#tokenizer: lowercase, alphabetic, apostrophes
token_pat = re.compile(r"[A-Za-z']+")

def tokenize_title(text: str):
    toks = [t.strip("'").lower() for t in token_pat.findall(text or "")]
    #keep only words present in GloVe vocab
    return [t for t in toks if t and t in kv.key_to_index]

def mean_pool(words):
    if not words:
        return np.zeros(EMBED_DIM, dtype=np.float32)
    vecs = [kv[w] for w in words if w in kv.key_to_index]
    return np.mean(vecs, axis=0).astype(np.float32) if vecs else np.zeros(EMBED_DIM, dtype=np.float32)

def cosine(u, v):
    u, v = np.asarray(u), np.asarray(v)
    nu, nv = norm(u), norm(v)
    return 0.0 if nu == 0 or nv == 0 else float(np.dot(u, v) / (nu * nv))


In [41]:
#build 100-D vectors from title and plot (mean of word vectors)
RECOMPUTE_ALL = True  # flip to True to refresh all vectors

conn = pymysql.connect(**DB_CONFIG)
with conn.cursor(pymysql.cursors.DictCursor) as cur:
    if RECOMPUTE_ALL:
        cur.execute("SELECT id, title, plot FROM movies;")
    else:
        cur.execute("""
            SELECT id, title, plot
            FROM movies
            WHERE vec_json IS NULL OR vec_json = '';
        """)
    rows = cur.fetchall()
    print(f"Found {len(rows)} movies to (re)compute (title + plot).")

    updates = []
    for r in rows:
        mid   = r["id"]
        #concatenate title + plot
        text  = f"{r['title'] or ''} {r['plot'] or ''}".strip()
        toks  = tokenize_title(text)   # reuse tokenizer
        vec   = mean_pool(toks)        # 100-d float32
        updates.append((json.dumps(vec.tolist()), mid))

    if updates:
        cur.executemany("UPDATE movies SET vec_json = %s WHERE id = %s;", updates)
        conn.commit()
        print(f"Updated {len(updates)} movie vectors (title + plot).")
    else:
        print("No updates needed.")
conn.close()

Found 30 movies to (re)compute (title + plot).
Updated 30 movie vectors (title + plot).


In [42]:
def top_movies_for_word(word: str, topn: int = 10):
    w = (word or "").lower().strip()
    if w not in kv.key_to_index:
        print(f"'{w}' not in GloVe vocabulary.")
        return []

    qv = kv[w]
    conn = pymysql.connect(**DB_CONFIG)
    with conn.cursor(pymysql.cursors.DictCursor) as cur:
        cur.execute("""
            SELECT id, title, plot, vec_json
            FROM movies
            WHERE vec_json IS NOT NULL AND vec_json <> '';
        """)
        rows = cur.fetchall()
    conn.close()

    scored = []
    for r in rows:
        vec = np.array(json.loads(r["vec_json"]), dtype=np.float32)
        scored.append((cosine(qv, vec), r["title"], r["plot"]))
    scored.sort(reverse=True, key=lambda x: x[0])
    return scored[:topn]

# test case to look for movies relevant to "love":
for s, title, plot in top_movies_for_word("love", 10):
    print(f"{s:.3f}  {title}  —  {plot[:70]}...")


0.773  Pride and Prejudice  —  A young woman challenges social class to find true love....
0.757  La La Land  —  A musician and an actress fall in love while chasing their dreams....
0.744  Me Before You  —  A young woman cares for a paralyzed man and they fall in love....
0.705  Crazy Rich Asians  —  A woman discovers her boyfriend is from a wealthy and powerful family....
0.703  A Star Is Born  —  A musician helps a young singer find fame as he declines....
0.692  The Holiday  —  Two women swap homes and find unexpected love....
0.683  Titanic  —  A love story unfolds aboard the doomed ocean liner....
0.677  Inception  —  A thief enters dreams to steal secrets from the subconscious....
0.673  The Fault in Our Stars  —  Two teens with cancer fall in love while facing mortality....
0.658  The Terminator  —  A cyborg assassin travels through time to kill a woman whose son will ...
